In [1]:
import torch
import numpy as np
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from torchinfo import summary
import numpy as np
import matplotlib.pyplot as plt
import pprint

from CharToIndex import CharToIndex
from MyDatasets import BaseDataset_set3 as MyDataset
from MyDatasets import Cross_Validation
from MyCustomLayer import TenHotEncodeLayer


import time
import math

In [2]:
chars_file_path = "/net/nfs2/export/home/ohno/CR_pytorch/data/tegaki_katsuji/all_chars_3812.npy"
tokens = CharToIndex(chars_file_path)
file_path = "/net/nfs2/export/home/ohno/CR_pytorch/data/tegaki_katsuji/tegaki.npy"
data = np.load(file_path,allow_pickle=True)

EMBEDDING_DIM = 10
HIDDEN_SIZE = 128
BATCH_SIZE = 64
VOCAB_SIZE = len(tokens)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tegaki_dataset = MyDataset(data,chars_file_path,device=device)

In [3]:
def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def show_ans_pred(answers,predictions):
    for ans,pred in zip(answers,predictions):
        correct = '✓' if ans.item() == pred.item() else '✗'
        print(f'{tokens.get_decoded_char(ans.item())}{tokens.get_decoded_char(pred.item()):2} {correct}',end=' ')
    print()



def train(model,train_dataloader,learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    batch_size = next(iter(train_dataloader))[0].size(0)
    running_loss = 0
    accuracy = 0

    model.train()
    for i,(x,y) in enumerate(train_dataloader):
        output = model(x)
        loss = criterion(output, y) #損失計算
        prediction = output.data.max(1)[1] #予測結果
        accuracy += prediction.eq(y.data).sum().item()/batch_size
        optimizer.zero_grad() #勾配初期化
        loss.backward(retain_graph=True) #逆伝播
        optimizer.step()  #重み更新
        running_loss += loss.item()

    loss_result = running_loss/len(train_dataloader)
    accuracy_result = accuracy/len(train_dataloader)

    return loss_result,accuracy_result


def eval(model,valid_dataloader,is_show_ans_pred=False):
    accuracy = 0
    batch_size = next(iter(valid_dataloader))[0].size(0)
    model.eval()
    for x,y in valid_dataloader:
        output = model(x)
        prediction = output.data.max(1)[1] #予測結果
        accuracy += prediction.eq(y.data).sum().item()/batch_size
        if is_show_ans_pred:
            ans_pred_list=show_ans_pred(y,prediction)
            print(ans_pred_list)

    return accuracy/len(valid_dataloader)




In [4]:
#hot encode用
class Proofreader(nn.Module):
    def __init__(self, input_size, hidden_dim, output_size,n_layers):
        super(Proofreader, self).__init__()

        self.output_size = output_size
        self.hidden_dim = hidden_dim
        self.n_layers  = n_layers
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        self.encoder = TenHotEncodeLayer(output_size)
        self.rnn = nn.RNN(output_size, self.hidden_dim, batch_first=True)
        self.fc = nn.Linear(self.hidden_dim, output_size)
        self.dropout = torch.nn.Dropout(p=0.5)
        self.to(self.device)

    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim)
        return hidden

    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size).to(self.device)
        x=self.encoder(x)
        out, hidden = self.rnn(x.float(), hidden)
        out = out[:,-1,:]
        out = self.dropout(out)
        out = self.fc(out)

        return out


In [5]:
def get_correct_char(model,valid_dataloader,correct_char):
    accuracy = 0
    batch_size = next(iter(valid_dataloader))[0].size(0)
    model.eval()
    for x,y in valid_dataloader:
        output = model(x)
        prediction = output.data.max(1)[1] #予測結果
        accuracy += prediction.eq(y.data).sum().item()/batch_size

        for correct,idx in zip(prediction.eq(y.data),y.data):
            if correct:
                correct_char[idx]+=1


    return accuracy/len(valid_dataloader),correct_char


In [6]:
final_accuracies = []
final_losses = []
correct_char=torch.zeros(len(tokens),dtype=torch.int)

cross_validation = Cross_Validation(tegaki_dataset)
k_num = cross_validation.k_num #デフォルトは10

##学習
for i in range(1):
    train_dataset,valid_dataset = cross_validation.get_datasets(k_idx=i)

    print(f'Cross Validation: k=[{i+1}/{k_num}]')
    train_dataloader=DataLoader(train_dataset,batch_size=BATCH_SIZE,shuffle=True,drop_last=True) #訓練データのみシャッフル
    valid_dataloader=DataLoader(valid_dataset,batch_size=BATCH_SIZE,shuffle=False,drop_last=True)
    model = Proofreader(VOCAB_SIZE, hidden_dim=HIDDEN_SIZE, output_size=VOCAB_SIZE, n_layers=1)
    # model.load_state_dict(torch.load("data/tegaki_katsuji/pre_trained_model.pth"))

    epochs = 100
    acc_record=[]
    loss_record=[]
    start = time.time() #開始時間の設定

    for epoch in range(1,epochs+1):
        loss,acc = train(model,train_dataloader,learning_rate=0.01)

        valid_acc = eval(model,valid_dataloader)
        loss_record.append(loss)
        acc_record.append(valid_acc)

        if epoch%10==0:
            print(f'epoch:[{epoch:3}/{epochs}] | {timeSince(start)} - loss: {loss:.7},  accuracy: {acc:.7},  valid_acc: {valid_acc:.7}')
            start = time.time() #開始時間の設定
        # print(f'epoch:[{epoch:3}/{epochs}] | {timeSince(start)} - loss: {loss:.7},  accuracy: {acc:.7},  valid_acc: {valid_acc:.7}')
        # start = time.time() #開始時間の設定
    acc,correct_char=get_correct_char(model,valid_dataloader,correct_char)


    print(f'final_loss: {loss_record[-1]:.7},   final_accuracy:{acc_record[-1]:.7}\n\n')
    final_accuracies.append(acc_record[-1])
    final_losses.append(loss_record[-1])

Cross Validation: k=[1/10]
final_loss: 3.122998,   final_accuracy:0.6711957




In [7]:
#正解した文字の集計
sorted_indices = torch.argsort(correct_char,descending=True)
for idx in sorted_indices:
    print(f'{tokens.get_decoded_char(idx.item())}: {correct_char[idx]}回')

print(f'losses: {final_losses}')
print(f'accuracies: {final_accuracies}')

の: 54回
に: 36回
､: 36回
る: 35回
い: 32回
と: 30回
た: 30回
し: 30回
｡: 27回
て: 27回
な: 23回
が: 22回
を: 22回
で: 20回
り: 18回
す: 18回
は: 18回
学: 15回
れ: 15回
こ: 14回
よ: 14回
も: 14回
研: 13回
あ: 12回
う: 11回
か: 11回
ま: 11回
そ: 10回
ら: 10回
く: 10回
さ: 10回
ｰ: 10回
き: 9回
間: 9回
究: 9回
つ: 7回
ﾝ: 7回
ど: 7回
人: 7回
合: 7回
え: 7回
お: 7回
場: 6回
わ: 6回
だ: 6回
的: 5回
通: 5回
者: 5回
用: 5回
(: 4回
ﾙ: 4回
け: 4回
神: 4回
後: 4回
): 4回
っ: 4回
所: 4回
報: 4回
や: 4回
基: 3回
知: 3回
せ: 3回
手: 3回
大: 3回
理: 3回
行: 3回
上: 3回
社: 3回
ﾒ: 3回
ん: 3回
時: 3回
発: 3回
ﾃ: 3回
業: 3回
見: 3回
ｽ: 3回
ば: 3回
性: 2回
験: 2回
出: 2回
要: 2回
高: 2回
続: 2回
興: 2回
ご: 2回
決: 2回
前: 2回
め: 2回
動: 2回
み: 2回
へ: 2回
本: 2回
期: 2回
金: 2回
ﾀ: 2回
度: 2回
ｸ: 2回
ﾌﾟ: 2回
明: 2回
重: 2回
ｲ: 2回
ﾄﾞ: 2回
日: 2回
化: 2回
認: 2回
講: 2回
使: 2回
取: 2回
客: 1回
ﾊﾟ: 1回
会: 1回
定: 1回
持: 1回
ﾌ: 1回
べ: 1回
自: 1回
提: 1回
考: 1回
代: 1回
生: 1回
産: 1回
世: 1回
進: 1回
割: 1回
法: 1回
ゆ: 1回
当: 1回
T: 1回
実: 1回
機: 1回
増: 1回
年: 1回
利: 1回
流: 1回
初: 1回
別: 1回
名: 1回
同: 1回
換: 1回
分: 1回
｣: 1回
歩: 1回
確: 1回
最: 1回
能: 1回
物: 1回
様: 1回
際: 1回
然: 1回
計: 1回
ﾗ: 1回
命: 1回
ざ: 1回
内: 1回
味: 1回
費: 1回
快: 1回
細: 1回
ﾑ: 1回
相: 1回
来: 1回